In [433]:
import csv
import pandas as pd
import numpy as np


In [434]:
df=pd.read_csv("BWFakeData.csv")
df.head(20)

,category,annualSpent,ads,aur,upt,numberDept,age,income,prevCategory
0,0,1765.654128,138.043031,51.375412,2.184242,8.059339,48.603100,5929.269951,0
1,0,1779.235457,141.101633,53.855448,1.792615,9.327111,51.877461,42232.922614,1
2,0,1791.259340,138.903619,52.842852,2.413194,13.692749,40.653785,7334.415152,0
3,0,1894.025873,143.090405,50.879810,3.718243,5.718444,44.801199,1476.881276,0
4,0,1754.563059,136.757503,49.715023,3.384003,13.361783,46.135527,2100.226851,0
5,0,1818.565878,140.828628,50.040162,2.404207,4.984910,45.814643,18724.175651,1
6,0,1775.553064,138.314849,51.097919,3.260138,8.313227,42.890914,2892.572459,1
7,0,1744.784080,143.378833,56.585813,1.843012,15.980275,42.170368,23304.702718,2
8,0,1791.998302,136.860752,51.000622,2.384818,9.841851,48.497411,1423.804713,1
9,0,1817.856526,133.350156,41.431236,1.579885,0.644297,46.682616,105103.659793,1


In [563]:
migrationsStacked = df.groupby(['prevCategory','category']).agg({'annualSpent' : 'count'})
migrations = migrationsStacked.unstack(level=-1)
migrations.columns= migrations.columns.droplevel()
migrations = migrations.fillna(0)
migrations

category,0,1,2,3,4
prevCategory,,,,,
0,24.0,52.0,53.0,6.0,3.0
1,21.0,86.0,136.0,32.0,22.0
2,4.0,29.0,147.0,83.0,47.0
3,0.0,3.0,57.0,93.0,42.0
4,1.0,0.0,7.0,29.0,26.0
5,0.0,0.0,0.0,7.0,10.0


In [564]:
migrationsToProspect = migrations.sum(axis=1)*0.06
migrations= pd.concat([migrations,migrationsToProspect ],axis=1)
migrations.columns = [0, 1, 2, 3, 4, 5]
migrationsPct = migrations.apply(lambda x: 100*x/float(x.sum()),axis=1).round(2)
print migrationsPct
migrationsPct.T.to_json()

                  0      1      2      3      4     5
prevCategory                                         
0             16.41  35.55  36.23   4.10   2.05  5.66
1              6.67  27.32  43.20  10.16   6.99  5.66
2              1.22   8.83  44.74  25.26  14.30  5.66
3              0.00   1.45  27.58  44.99  20.32  5.66
4              1.50   0.00  10.48  43.43  38.93  5.66
5              0.00   0.00   0.00  38.85  55.49  5.66


'{"0":{"0":16.41,"1":35.55,"2":36.23,"3":4.1,"4":2.05,"5":5.66},"1":{"0":6.67,"1":27.32,"2":43.2,"3":10.16,"4":6.99,"5":5.66},"2":{"0":1.22,"1":8.83,"2":44.74,"3":25.26,"4":14.3,"5":5.66},"3":{"0":0.0,"1":1.45,"2":27.58,"3":44.99,"4":20.32,"5":5.66},"4":{"0":1.5,"1":0.0,"2":10.48,"3":43.43,"4":38.93,"5":5.66},"5":{"0":0.0,"1":0.0,"2":0.0,"3":38.85,"4":55.49,"5":5.66}}'

## Migration value impact

In [437]:

catAnnualSpent = df.groupby('category').sum()['annualSpent']
catAnnualSpent[5]=0
annualSpentAvg = catAnnualSpent/migrationsPct.sum(axis=0)
annualSpentAvg[5] = 0
annualSpentAvg


category
0    3480.781628
1    1392.041119
2     590.798680
3     169.120763
4      34.311947
5       0.000000
dtype: float64

In [438]:
prospectPool = 10000
catCount = df.groupby('category').count()['annualSpent']
catCount[5]= prospectPool - catCount.sum()
catCount

category
0      50
1     170
2     400
3     250
4     150
5    8980
Name: annualSpent, dtype: int64

In [439]:
50*16.41+170*6.67+400*1.22+150*1.5
93.026*16.41+123.937*6.67+1798.5253*1.22+3151.768*1.5

9275.069316000001

In [440]:
catCount2 = (catCount.values).dot(migrations.values/100)
catCount3 = (catCount2).dot(migrations.values/100)

print catCount2
print catCount3


[   26.674   103.164   355.185  3786.712  5162.305   566.   ]
[   93.0260742   123.9371713  1798.5253408  4266.8166177  3151.7680505
   566.002264 ]


In [441]:
# Matrix multiplication
catAnnualSpentEvolution=[]
catAnnualSpentEvolution.append(annualSpentAvg*catCount)
prevCount = (catCount.values).dot(migrations.values/100)
catAnnualSpentEvolution.append(annualSpentAvg*prevCount)

for i in range(10):
    prevCount = prevCount.dot(migrations.values/100)
    catAnnualSpentEvolution.append(annualSpentAvg*prevCount)
pd.DataFrame(catAnnualSpentEvolution).sum(axis=1)

0     6.944325e+05
1     1.263838e+06
2     2.388646e+06
3     2.905288e+06
4     3.164553e+06
5     3.287864e+06
6     3.344574e+06
7     3.370260e+06
8     3.381838e+06
9     3.387068e+06
10    3.389456e+06
11    3.390573e+06
dtype: float64

In [442]:
catAnnualSpentEvoSum = pd.DataFrame(catAnnualSpentEvolution).sum(axis=1)
catAnnualSpentEvoSum2 = catAnnualSpentEvoSum
newdf = pd.DataFrame({"Default":catAnnualSpentEvoSum,"Alt1":catAnnualSpentEvoSum2,"Year": [i for i in range(12)]})
newdf.to_json(orient="records")

'[{"Alt1":694432.5262306527,"Default":694432.5262306527,"Year":0},{"Alt1":1263838.0868760042,"Default":1263838.0868760042,"Year":1},{"Alt1":2388646.0652692299,"Default":2388646.0652692299,"Year":2},{"Alt1":2905287.9083062471,"Default":2905287.9083062471,"Year":3},{"Alt1":3164552.8038807414,"Default":3164552.8038807414,"Year":4},{"Alt1":3287863.5232287655,"Default":3287863.5232287655,"Year":5},{"Alt1":3344573.6988711869,"Default":3344573.6988711869,"Year":6},{"Alt1":3370260.2633025539,"Default":3370260.2633025539,"Year":7},{"Alt1":3381838.1816199538,"Default":3381838.1816199538,"Year":8},{"Alt1":3387068.407653986,"Default":3387068.407653986,"Year":9},{"Alt1":3389456.0794891855,"Default":3389456.0794891855,"Year":10},{"Alt1":3390573.4055621144,"Default":3390573.4055621144,"Year":11}]'

## Sankey data formating

In [586]:
#catList = df["prevCategory"].unique()
#catList = np.append(catList, ["N"+str(i) for i in df["category"].unique()])
nodesList=[0,1,2,3,4,5,"N0","N1","N2","N3","N4","N5"]
#for i in range(len(catList)):
#        nodesList.append({"name":catList[i]})
print nodesList
migrationsStacked2 = migrations.stack()
connections=[]
for i in range(6):
    for j in range(6):
        try:
            value = migrationsStacked2.loc[i,j]
        except TypeError,IndexError:
            value = 0
        connections.append({"source":i,"target":6+j,"value":round(value)})
connections

[0, 1, 2, 3, 4, 5, 'N0', 'N1', 'N2', 'N3', 'N4', 'N5']


[{'source': 0, 'target': 6, 'value': 24.0},
 {'source': 0, 'target': 7, 'value': 52.0},
 {'source': 0, 'target': 8, 'value': 53.0},
 {'source': 0, 'target': 9, 'value': 6.0},
 {'source': 0, 'target': 10, 'value': 3.0},
 {'source': 0, 'target': 11, 'value': 8.0},
 {'source': 1, 'target': 6, 'value': 21.0},
 {'source': 1, 'target': 7, 'value': 86.0},
 {'source': 1, 'target': 8, 'value': 136.0},
 {'source': 1, 'target': 9, 'value': 32.0},
 {'source': 1, 'target': 10, 'value': 22.0},
 {'source': 1, 'target': 11, 'value': 18.0},
 {'source': 2, 'target': 6, 'value': 4.0},
 {'source': 2, 'target': 7, 'value': 29.0},
 {'source': 2, 'target': 8, 'value': 147.0},
 {'source': 2, 'target': 9, 'value': 83.0},
 {'source': 2, 'target': 10, 'value': 47.0},
 {'source': 2, 'target': 11, 'value': 19.0},
 {'source': 3, 'target': 6, 'value': 0.0},
 {'source': 3, 'target': 7, 'value': 3.0},
 {'source': 3, 'target': 8, 'value': 57.0},
 {'source': 3, 'target': 9, 'value': 93.0},
 {'source': 3, 'target': 10, '

In [590]:
#circular network

#catList = df["prevCategory"].unique()
#catList = np.append(catList, ["N"+str(i) for i in df["category"].unique()])
catList=[0,1,2,3,4,5]
nodesList= []
for i in range(len(catList)):
        nodesList.append({"name":catList[i]})
print nodesList

migrationsStacked2 = migrations.stack()
connections=[]
for i in range(6):
    for j in range(6):
        try:
            value = migrationsStacked2.loc[i,j]
        except TypeError,IndexError:
            value = 0
        connections.append(["cat"+str(i),"cat"+str(j),round(value)])
connections

[{'name': 0}, {'name': 1}, {'name': 2}, {'name': 3}, {'name': 4}, {'name': 5}]


[['cat0', 'cat0', 24.0],
 ['cat0', 'cat1', 52.0],
 ['cat0', 'cat2', 53.0],
 ['cat0', 'cat3', 6.0],
 ['cat0', 'cat4', 3.0],
 ['cat0', 'cat5', 8.0],
 ['cat1', 'cat0', 21.0],
 ['cat1', 'cat1', 86.0],
 ['cat1', 'cat2', 136.0],
 ['cat1', 'cat3', 32.0],
 ['cat1', 'cat4', 22.0],
 ['cat1', 'cat5', 18.0],
 ['cat2', 'cat0', 4.0],
 ['cat2', 'cat1', 29.0],
 ['cat2', 'cat2', 147.0],
 ['cat2', 'cat3', 83.0],
 ['cat2', 'cat4', 47.0],
 ['cat2', 'cat5', 19.0],
 ['cat3', 'cat0', 0.0],
 ['cat3', 'cat1', 3.0],
 ['cat3', 'cat2', 57.0],
 ['cat3', 'cat3', 93.0],
 ['cat3', 'cat4', 42.0],
 ['cat3', 'cat5', 12.0],
 ['cat4', 'cat0', 1.0],
 ['cat4', 'cat1', 0.0],
 ['cat4', 'cat2', 7.0],
 ['cat4', 'cat3', 29.0],
 ['cat4', 'cat4', 26.0],
 ['cat4', 'cat5', 4.0],
 ['cat5', 'cat0', 0.0],
 ['cat5', 'cat1', 0.0],
 ['cat5', 'cat2', 0.0],
 ['cat5', 'cat3', 7.0],
 ['cat5', 'cat4', 10.0],
 ['cat5', 'cat5', 1.0]]

In [583]:
migrations

,0,1,2,3,4,5
prevCategory,,,,,,
0,24.0,52.0,53.0,6.0,3.0,8.28
1,21.0,86.0,136.0,32.0,22.0,17.82
2,4.0,29.0,147.0,83.0,47.0,18.60
3,0.0,3.0,57.0,93.0,42.0,11.70
4,1.0,0.0,7.0,29.0,26.0,3.78
5,0.0,0.0,0.0,7.0,10.0,1.02
